In [1]:
# use OOT data to estimate bkg in signal region
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections
import os
from collections import OrderedDict
import uproot
import pandas as pd
import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0
wH = 1
Z_MASS = 91.2
# donotdelete = []
print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [2]:
fpath =OrderedDict()
tree = OrderedDict()
old_ctau = OrderedDict()
# mass = [15]
# VBFH = 0
# decay = 'dddd'

# mass = [7, 15, 40, 55]
# VBFH=1
# decay = 'bbbb'
lumi = 137000
ntupler_version = 'V1p17/'
analyzer_version = 'v1/v9'
data_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/' + ntupler_version + '/Data2018/v5/v9/normalized/'
sig_path = '/mnt/hadoop/store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/' + ntupler_version +\
'/MC_Fall18/' + analyzer_version + '/normalized/'

fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'
fpath['MC_ggH_HToSS_SToEE_ms0p1_pl500'] = sig_path +'ggH_HToSS_SToEE_ms0p1_pl500_1pb_weighted.root' 
fpath['MC_ggH_HToSS_SToPi0Pi0_ms1_pl500'] = sig_path + 'ggH_HToSS_SToPi0Pi0_ms1_pl500_1pb_weighted.root'
fpath['MC_ggH_HToSS_SToPiPlusPiMinus_ms1_pl500'] = sig_path + 'ggH_HToSS_SToPiPlusPiMinus_ms1_pl500_1pb_weighted.root'
fpath['MC_ggH_HToSS_SToKPlusKMinus_ms1p5_pl500'] = sig_path + 'ggH_HToSS_SToKPlusKMinus_ms1p5_pl500_1pb_weighted.root'
fpath['MC_ggH_HToSSTobbbb_ms1_pl1000'] = sig_path +  'ggH_HToSSTobbbb_ms1_pl1000_1pb_weighted.root'
fpath['MC_ggH_HToSSTobbbb_MH-125_MS-15_ctau-1000'] = sig_path +  'ggH_HToSSTobbbb_MH-125_MS-15_ctau-1000_TuneCP5_13TeV-powheg-pythia8_1pb_weighted.root'
fpath['MC_ggH_HToSS_SToEE_ms0p4_pl500'] = sig_path + 'ggH_HToSS_SToEE_ms0p4_pl500_1pb_weighted.root'
fpath['MC_ggH_HToSSTo4Tau_MH-125_MS-7_ctau-1000'] = sig_path +  'ggH_HToSSTo4Tau_MH-125_MS-7_ctau-1000_TuneCP5_13TeV-powheg-pythia8_1pb_weighted.root'
old_ctau['MC_ggH_HToSS_SToEE_ms0p4_pl500'] = 500
old_ctau['MC_ggH_HToSSTobbbb_MH-125_MS-15_ctau-1000'] = 1000
old_ctau['MC_ggH_HToSSTo4Tau_MH-125_MS-7_ctau-1000'] = 1000
old_ctau['MC_ggH_HToSSTobbbb_ms1_pl1000'] = 1000
old_ctau['MC_ggH_HToSS_SToKPlusKMinus_ms1p5_pl500'] = 500
old_ctau['MC_ggH_HToSS_SToPiPlusPiMinus_ms1_pl500'] = 500
old_ctau['MC_ggH_HToSS_SToPi0Pi0_ms1_pl500'] = 500
old_ctau['MC_ggH_HToSS_SToEE_ms0p1_pl500'] = 500

NEvents = {}
NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    w = tree[k]["weight"].array()

In [3]:
bdt_name = "cut_based_v4"

# nCsc with different hit vetoing

In [4]:
%%time

start_t = time.time()
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
ring_cut = 50
if bdt_name == 'bdt_flatten_metfilter_me1112nohits_eta2p0_testsize0p2_noSpread_v2':
    BDT_CUT = 0.2983932 
    bdtBkgEff = 0.34527687296416937
    BDT_BINS = [0.0, BDT_CUT]
elif "cut_based" in bdt_name:
    print("cut based")
else:
    print('BDT NAME NOT FOUND')
DPHI_CUT = 1
weight = {}
weight_event = {}
weight_ctau = {}
lumiSec = {}
evtNum = {}
nCsc = {}
npv = {}
npu = {}
runNum = {}
nCsc_JetMuonVetoCluster0p4_Me1112Veto = {}
gLLP_csc = {}
cscClusterSize = {}
met = {}
pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
cscRechitClusterTimeDiff = {}
cscRechitCluster_match_gLLP = {}
higgsPtWeight = {}
cscRechitClusterNStation = {}
cscRechitClusterAvgStation = {}
cscRechitClusterNStation10  = {}
cscRechitClusterAvgStation10 = {}
cscRechitClusterEta = {}
jetMet_dPhiMin30 = {}
dphiMet_cluster = {}
jetMet_dPhiMin30_sr = {}
sel_ev = {}
bdt_sel = {}
sf_facScaleUp = {}
sf_facScaleDown = {}
sf_renScaleUp = {}
sf_renScaleDown = {}
sf_facRenScaleUp = {}
sf_facRenScaleDown = {}
cluster_index = '3'

for k in list(tree.keys()) + ['data_oot_sr', 'data_oot_vr', 'data_intime_vr']:
########### SELECTION: CLUSTERS ############
    if 'data' in k: T = tree['data']
    else: T = tree[k]
    sel_rechitcluster = np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta')) < 2.0
    me1112_veto = 0
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_MB1Seg_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RE12_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + '_match_RB1_0p4') <= me1112_veto)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'TimeSpread') <= 20)
    if 'oot' in k:
        sel_rechitcluster = np.logical_and(sel_rechitcluster,  T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < -12.5)
    else:
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < JET_PT_CUT)
        sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT)
        sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeTotal') < 12.5, T.array('cscRechitCluster' + cluster_index + 'TimeTotal') > -5.0))
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') < JET_PT_CUT)
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt') < MUON_PT_CUT)
    print(np.count_nonzero(sel_rechitcluster.flatten()))

########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 )

########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()
    sel_ev[k]  = hlt[:,310]
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('met') > 200)
    sel_ev[k]  = np.logical_and(sel_ev[k], T.array('category') == 0)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('nLeptons') == 0)
    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    sel_ev[k]  = np.logical_and(sel_ev[k],sel_rechitcluster.sum() == 1)
    sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
    gLLP_csc[k] = np.sum(T.array('gLLP_csc'),axis = 1)
    if 'MC' in k:
        sel_ev[k] = np.logical_and(sel_ev[k], np.sum(T.array('gLLP_csc'),axis = 1) > 0)

########### BRANCHES ############
    
    ##### bdt variables ####
    cscRechitClusterNStation[k] = T.array('cscRechitCluster' + cluster_index + 'NStation5')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterNStation10[k] = T.array('cscRechitCluster' + cluster_index + 'NStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterAvgStation[k] = T.array('cscRechitCluster' + cluster_index + 'AvgStation5')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterAvgStation10[k] = T.array('cscRechitCluster' + cluster_index + 'AvgStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    cscRechitClusterEta[k] = T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    if len(cscRechitClusterNStation[k])>0:
        if bdt_name == 'cut_based_v4':
            cond2 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==2, np.abs(cscRechitClusterEta[k]) < 1.6)
            cond3 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==3, np.abs(cscRechitClusterEta[k]) < 1.6)
            cond4 = np.logical_and(np.abs(cscRechitClusterAvgStation10[k])==4, np.abs(cscRechitClusterEta[k]) < 1.8)
            cond1 = np.logical_and(cscRechitClusterNStation10[k]==1, np.logical_or(np.logical_or(np.abs(cscRechitClusterAvgStation10[k])==1, cond2), np.logical_or(cond3, cond4)))
            cond2 = np.logical_and(cscRechitClusterNStation10[k] > 1, np.abs(cscRechitClusterEta[k]) < 1.9)
            bdt_sel[k] = np.logical_or(np.logical_or(cond1, cond2), np.logical_or(cond3, cond4))
            if 'vr' in k: bdt_sel[k] = np.logical_not(bdt_sel[k])
        else:
            print("CUT BASED ERROR")
            
        print(k, "effiency",np.count_nonzero(bdt_sel[k])/len(bdt_sel[k]))

        dphiMet_cluster[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()[bdt_sel[k]]
        jetMet_dPhiMin30[k] = T.array('jetMet_dPhiMin')[sel_ev[k]][bdt_sel[k]]
        weight[k] = T.array('weight')[sel_ev[k]][bdt_sel[k]]
        npv[k] = T.array('npv')[sel_ev[k]][bdt_sel[k]]
        if k[:2] == 'MC':
            pileupWeight[k] = T.array('pileupWeight')[sel_ev[k]][bdt_sel[k]]
            higgsPtWeight[k] = T.array('higgsPtWeight')[sel_ev[k]][bdt_sel[k]]
            sf_facScaleUp[k] = T.array('sf_facScaleUp')[sel_ev[k]][bdt_sel[k]]
            sf_facScaleDown[k] = T.array('sf_facScaleDown')[sel_ev[k]][bdt_sel[k]]
            sf_renScaleUp[k] = T.array('sf_renScaleUp')[sel_ev[k]][bdt_sel[k]]
            sf_renScaleDown[k] = T.array('sf_renScaleDown')[sel_ev[k]][bdt_sel[k]]
            sf_facRenScaleUp[k] = T.array('sf_facRenScaleUp')[sel_ev[k]][bdt_sel[k]]
            sf_facRenScaleDown[k] = T.array('sf_facRenScaleDown')[sel_ev[k]][bdt_sel[k]]
        else:
            pileupWeight[k] = np.ones(weight[k].shape, dtype=bool)
            higgsPtWeight[k] = np.ones(weight[k].shape, dtype=bool)
            sf_facScaleUp[k] = np.ones(weight[k].shape, dtype=bool)
            sf_facScaleDown[k] = np.ones(weight[k].shape, dtype=bool)
            sf_renScaleUp[k] = np.ones(weight[k].shape, dtype=bool)
            sf_renScaleDown[k] = np.ones(weight[k].shape, dtype=bool)
            sf_facRenScaleUp[k] = np.ones(weight[k].shape, dtype=bool)
            sf_facRenScaleDown[k] = np.ones(weight[k].shape, dtype=bool)
        if k[:2] == 'MC':
            weight[k] = weight[k]*lumi
        cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_rechitcluster][sel_ev[k]][bdt_sel[k]]
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = cscClusterSize[k][:,0]
    else:
        dphiMet_cluster[k] = np.abs(T.array('cscRechitCluster' + cluster_index + 'Met_dPhi'))[sel_rechitcluster][sel_ev[k]][:,0].flatten()
        weight[k] = T.array('weight')[sel_ev[k]]
        npv[k] = T.array('npv')[sel_ev[k]]
        if k[:2] == 'MC':
            pileupWeight[k] = T.array('pileupWeight')[sel_ev[k]]
        else:
            pileupWeight[k] = np.ones(weight[k].shape, dtype=bool)
        if k[:2] == 'MC':
            weight[k] = weight[k]*lumi
        cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_rechitcluster][sel_ev[k]]
        nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] = cscClusterSize[k][:,0]
        print("no BDT applied")

cut based
9941
data effiency 0.29366602687140114
117
MC_ggH_HToSS_SToEE_ms0p1_pl500 effiency 0.83
110
MC_ggH_HToSS_SToPi0Pi0_ms1_pl500 effiency 0.8817204301075269
205
MC_ggH_HToSS_SToPiPlusPiMinus_ms1_pl500 effiency 0.9058823529411765
249
MC_ggH_HToSS_SToKPlusKMinus_ms1p5_pl500 effiency 0.9205607476635514
106
MC_ggH_HToSSTobbbb_ms1_pl1000 effiency 0.8865979381443299
1667
MC_ggH_HToSSTobbbb_MH-125_MS-15_ctau-1000 effiency 0.9195979899497487
58
MC_ggH_HToSS_SToEE_ms0p4_pl500 effiency 0.8571428571428571
988
MC_ggH_HToSSTo4Tau_MH-125_MS-7_ctau-1000 effiency 0.8850855745721271
13124
data_oot_sr effiency 0.32528504359490273
13124
data_oot_vr effiency 0.6747149564050973
9941
data_intime_vr effiency 0.7063339731285988
CPU times: user 49.9 s, sys: 4.89 s, total: 54.8 s
Wall time: 1min 32s


# life time reweight

In [5]:
def weight_calc(llp_ct, new_ctau, old_ctau):
    source = np.exp(-1.0*llp_ct/old_ctau)/old_ctau**2
    weight = 1.0/new_ctau**2 * np.exp(-1.0*llp_ct/new_ctau)/source
    return weight

# signal yield summary table

In [6]:
ctaus = ['50', '100', '500']
BR = 0.01
N_RECHIT_CUT = 130
DPHI_CUT = 0.75      
var = dphiMet_cluster
for N_RECHIT_CUT in np.arange(60,220,10):
    if not N_RECHIT_CUT== 130:continue
    for k in fpath.keys():
        if k == 'data':continue
        signal_rate = []
        unc_rate = []
        for ct in ctaus:   
            T = tree[k]
            gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
            if len(gLLP_ctau) == 0: continue
            weight_ctau = weight_calc(gLLP_ctau, float(ct)/10, old_ctau[k]/10) # convert everything to cm
            w = pileupWeight[k] * weight[k] * weight_ctau * higgsPtWeight[k]
            cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT)
            signal_rate.append(np.sum(w[cond]))
        signal_rate = np.array(signal_rate)
        print(N_RECHIT_CUT, '\t', signal_rate*BR)

130 	 [8.40250201 8.21504909 2.73673316]
130 	 [8.75538566 8.83315917 2.90749692]
130 	 [14.17254653 13.8102358   5.90590637]
130 	 [20.63312417 17.82346115  8.42351777]
130 	 [11.33924911 13.36205253  5.42943824]
130 	 [ 0.95268156  5.89833585 22.74223541]
130 	 [3.81886442 3.77990261 1.29055732]
130 	 [ 5.14850033 12.2400472  14.39226585]


# make datacard

In [8]:
import os
N_RECHIT_CUTS = np.arange(60,220, 10)

ctaus = ['5','10','20', '30','40', '50','60','80', '100','200','300','500', '1000', '5000'] #mm
datacard_version  = 'v1'
bkg_unc = 1.0
sig_unc0 = 0.2

var = dphiMet_cluster
DPHI_CUTS = [0.75]

for N_RECHIT_CUT in N_RECHIT_CUTS:
    for DPHI_CUT in DPHI_CUTS:
        ##### method B #####
        factor = len(var['data_intime_vr'])/len(var['data_oot_vr'])
        k = 'data_oot_sr'
        a = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] >= DPHI_CUT))
        b = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] >= DPHI_CUT)) 
        c = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] < N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        d = np.count_nonzero(np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k] >= N_RECHIT_CUT, var[k] < DPHI_CUT)) 
        pred = a*c/b
        stat_unc = (1./a + 1./c + 1./b)**0.5*(pred)
        sys_unc = abs(d-pred)
        unc = [a**0.5/a, b**0.5/b, c**0.5/c, (stat_unc**2 + sys_unc**2)**0.5/pred]
        a = a * factor
        b = b * factor
        c = c * factor
        d = d * factor
        pred = pred * factor
        observation = np.array([a,b,c,pred])
        print(N_RECHIT_CUT, pred)
        
        #####
        # a = c*c1*c2
        # b = c1* c
        # c = c
        # d = c2*c
        #####

        outDataCardsDir = os.environ['HOME']+'/login-1/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine_gdituri/datacards/'+\
        ntupler_version+analyzer_version+'/'+ bdt_name+'/'+datacard_version+'/'
       # open(outDataCardsDir)
        if not os.path.isdir(outDataCardsDir):
            os.makedirs(outDataCardsDir)
            print("made new directory: ",outDataCardsDir)
        else:
            print("directory exists!", outDataCardsDir)
        sig_norm = []
        for k in fpath.keys():
            if k == 'data':continue
            for ct in ctaus:
                dphi = str(DPHI_CUT).replace(".", "p")
                modelName = k[3:]
                modelName = modelName + '_ctau'+str(ct)+'mm_nRechit'+str(N_RECHIT_CUT)+'dPhiCluster'+dphi
                signal_rate = np.zeros((4,))
                sig_unc =  np.zeros((4,))

                T = tree[k]
                gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][bdt_sel[k]]
                if len(gLLP_ctau) == 0: continue

                weight_ctau = weight_calc(gLLP_ctau, float(ct)/10, old_ctau[k]/10) # convert everything to cm
                w = pileupWeight[k] * weight[k] * weight_ctau * higgsPtWeight[k]

                cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]>=DPHI_CUT)
                signal_rate[0] =np.sum(w[cond])
                cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]>=DPHI_CUT)
                signal_rate[1]=np.sum(w[cond])
                cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]<N_RECHIT_CUT, var[k]<DPHI_CUT)
                signal_rate[2]=np.sum(w[cond])
                cond = np.logical_and(nCsc_JetMuonVetoCluster0p4_Me1112Veto[k]>=N_RECHIT_CUT, var[k]<DPHI_CUT)
                signal_rate[3]=np.sum(w[cond])

                norm = np.sum(signal_rate)
                sig_norm.append(norm)
                make_datacard(outDataCardsDir, modelName, False, signal_rate/norm, norm, observation, bkg_unc, np.array([sig_unc0, sig_unc0, sig_unc0, sig_unc0]))


60 18.480063018117708
directory exists! /storage/user/gdituri/login-1/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine_gdituri/datacards/V1p17/v1/v9/cut_based_v4/v1/
70 10.209173031983537
directory exists! /storage/user/gdituri/login-1/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine_gdituri/datacards/V1p17/v1/v9/cut_based_v4/v1/
80 5.480385976821995
directory exists! /storage/user/gdituri/login-1/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine_gdituri/datacards/V1p17/v1/v9/cut_based_v4/v1/
90 3.6158839470382316
directory exists! /storage/user/gdituri/login-1/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine_gdituri/datacards/V1p17/v1/v9/cut_based_v4/v1/
100 2.893374217969022
directory exists! /storage/user/gdituri/login-1/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine_gdituri/datacards/V1p17/v1/v9/cut_based_v4/v1/
110 1.5991482797205838
directory exists! /storage/user/gdituri/login-1/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine_gdituri/d

In [7]:
def make_datacard(outDataCardsDir,modelName, jetmet, signal_rate, norm, observation, unc, sig_unc):
    a,b,c,d = observation[0], observation[1], observation[2], observation[3]
    if jetmet:
        c1 = b/c
        c2 = d/c
    else:
        c1 = a/b
        c2 = c/b
    text_file = open(outDataCardsDir+modelName+".txt", "w")
    text_file.write('# signal norm {0} \n'.format(norm))

    text_file.write('imax {0} \n'.format(4))
    text_file.write('jmax {0} \n'.format(1))
    text_file.write('kmax * \n')
    text_file.write('--------------- \n')
    text_file.write('--------------- \n')
    text_file.write('bin \t chA \t chB \t chC \t chD \n')
    text_file.write('observation \t {0:6.2f} \t {1:6.2f} \t {2:6.2f} \t {3:6.2f} \n'.format(a, b, c, d))
    text_file.write('------------------------------ \n')  
    text_file.write('bin \t chA \t chA \t chB \t chB \t chC \t chC \t chD \t chD \n')
    text_file.write('process sig \t bkg \t sig \t bkg \t sig \t bkg \t sig \t bkg \n')
    text_file.write('process 0 \t 1 \t 0 \t 1 \t 0 \t 1 \t 0 \t 1 \n')
    rate_string = 'rate'
    for i, rate in enumerate(signal_rate):
        rate_string =  rate_string+'\t {0:e} \t 1'.format(rate)
    text_file.write(rate_string+'\n')
    text_file.write('------------------------------ \n')  
    for ch in ['A','B','C','D']:
        text_file.write('NA\t rateParam \t ch{0} \t bkg \t {1:6.2f} \n'.format(ch, c))
    if jetmet:
        text_file.write('c1\t rateParam \t chA \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chA \t bkg \t {0:e} \n'.format(c2))
        text_file.write('c1\t rateParam \t chB \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chD \t bkg \t {0:e} \n'.format(c2))
    else:
        text_file.write('c1\t rateParam \t chD \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chD \t bkg \t {0:e} \n'.format(c2))
        text_file.write('c1\t rateParam \t chA \t bkg \t {0:e} \n'.format(c1))
        text_file.write('c2\t rateParam \t chC \t bkg \t {0:e} \n'.format(c2))
#     #### uncertainties ####
    text_file.write('lumi\t lnN \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \t 1.025000 \t - \n')
    text_file.write('signal_yield \t lnN  \t {0} \t - \t {1} \t - \t {2} \t - \t {3} \t -\n'.format(1+sig_unc[0], 1+sig_unc[1], 1+sig_unc[2], 1+sig_unc[3]))    
#     if jetmet:
#         text_file.write('ABCD_closure \t lnN \t - \t {0} \t - \t {1} \t - \t {2} \t - \t {3} \n'.format(1+unc[0], 1+unc[1], 1+unc[2], 1+unc[3]))
#     else:
#         text_file.write('ABCD_closure \t lnN \t - \t {0} \t - \t {1} \t - \t {2} \t - \t {3} \n'.format(1+unc[0], 1+unc[1], 1+unc[2], 1+unc[3]))

    if jetmet:
        text_file.write('ABCD_closure \t lnN \t - \t {0} \t - \t - \t - \t - \t - \t - \n'.format(1+unc))
    else:
        text_file.write('ABCD_closure \t lnN \t - \t - \t - \t - \t - \t - \t - \t {0} \n'.format(1+unc))

    
    text_file.close()
